In [1]:
import os

import httpx
from retry import retry
from dotenv import load_dotenv


load_dotenv()

True

In [2]:
import json


def write_json(data, filename='data.json'):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

In [4]:
@retry(Exception, tries=3, delay=1, backoff=2, jitter=3)
def create_token_set(network: str, tokens: list[str]):
    networks = {"ethereum": "api", "polygon": "api-polygon"}

    url = f"https://{networks[network]}.reservoir.tools/token-sets/v2"

    headers = {
        "accept": "*/*",
        "content-type": "application/json",
        "x-api-key": os.getenv("RESERVOIR_API_KEY"),
    }
    body = {
        "tokens": tokens,
    }

    resp = httpx.post(url, json=body, headers=headers, timeout=30)

    # 200번 이외 경우에 에러를 반환하여 재시도를 하도록 함.
    resp.raise_for_status()

    resp = resp.json()
    
    return resp

In [5]:
tokens = [
    "0xd774557b647330c91bf44cfeab205095f7e6c367:1",
    "0xd774557b647330c91bf44cfeab205095f7e6c367:2",
]

resp = create_token_set("ethereum", tokens)
resp

{'id': 'list:0xd774557b647330c91bf44cfeab205095f7e6c367:0x2a171b5bcd1449348c3e09a5424946b5e6d6f5471221941d585131d673952ee4'}

In [9]:
token_set_id = resp["id"]

> 토큰 수량 제한이 나와있지 않아서, 민팅된 토큰을 전체 넣고, 1000건이 넘는 신규 민팅의 경우, continuation으로 추가 호출을 진행한다

In [8]:
@retry(Exception, tries=3, delay=1, backoff=2, jitter=3)
def get_tokens_by_token_set(network: str, token_set_id: str, continuation: str = ""):
    networks = {"ethereum": "api", "polygon": "api-polygon"}

    url_tokens = f"https://{networks[network]}.reservoir.tools/tokens/v7"

    headers = {
        "accept": "*/*",
        "content-type": "application/json",
        "x-api-key": os.getenv("RESERVOIR_API_KEY"),
    }
    params = {
        "tokenSetId": token_set_id,
        "limit": 1000,
        "sortBy": "updatedAt",
        "includeAttributes": True,
    }

    if continuation != "":
        params["continuation"] = continuation

    resp = httpx.get(url_tokens, params=params, headers=headers, timeout=30)

    # 200번 이외 경우에 에러를 반환하여 재시도를 하도록 함.
    resp.raise_for_status()

    tokens = resp.json()

    return tokens

In [10]:
resp = get_tokens_by_token_set("ethereum", token_set_id)
resp

{'tokens': [{'token': {'chainId': 1,
    'contract': '0xd774557b647330c91bf44cfeab205095f7e6c367',
    'tokenId': '1',
    'name': 'Nakamigos #1',
    'description': None,
    'image': 'https://img.reservoir.tools/images/v2/mainnet/i9YO%2F4yHXUdJsWcTqhqvf%2BcBP4JKdTmU0wHXWJZ7xwRCJ5z5Hr4MOmMzcCq7ZlUod3fillwZ9rybYihwvi2bPwy7LEcXMxHnCow9fIyJax5gnzzAa3eJMnovu2jMEdvE.png?width=512',
    'imageSmall': 'https://img.reservoir.tools/images/v2/mainnet/i9YO%2F4yHXUdJsWcTqhqvf%2BcBP4JKdTmU0wHXWJZ7xwRCJ5z5Hr4MOmMzcCq7ZlUod3fillwZ9rybYihwvi2bPwy7LEcXMxHnCow9fIyJax5gnzzAa3eJMnovu2jMEdvE.png?width=250',
    'imageLarge': 'https://img.reservoir.tools/images/v2/mainnet/i9YO%2F4yHXUdJsWcTqhqvf%2BcBP4JKdTmU0wHXWJZ7xwRCJ5z5Hr4MOmMzcCq7ZlUod3fillwZ9rybYihwvi2bPwy7LEcXMxHnCow9fIyJax5gnzzAa3eJMnovu2jMEdvE.png?width=1000',
    'metadata': {'imageOriginal': 'ipfs://QmVbye61dgnVGC2H2fE1obBvvE8zGXUD835J2vQU4KdE2F',
     'imageMimeType': 'image/png',
     'tokenURI': 'https://ipfs.io/ipfs/QmaN1jRPtmzeqhp6s3mR1SRK4